In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog, filedialog
import threading
import cv2
import torch
import time
from PIL import Image, ImageTk
import numpy as np
import pathlib
import subprocess
import os
import numpy as np  # To calculate average brightness


class VideoStreamApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Multi-RTSP Stream Monitor")

        # Initialize list of RTSP streams and their AI status
        self.rtsp_urls = []
        self.ai_status = []  # List to keep track of AI detection for each RTSP stream
        self.stream_running = []  # List to track if each stream is running
        self.save_paths = []  # List to store save paths for each stream

        # RTSP file to save the URLs
        self.rtsp_file = "rtsp_links.txt"

        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Set video dimensions
        self.video_width = int(screen_width / 5) - 5
        self.video_height = int(screen_height / 5) - 10

        # Frame for displaying video streams and controls
        self.video_frame = tk.Frame(root)
        self.video_frame.pack(side=tk.TOP, padx=5, pady=5)

        # Control frame for inputs
        self.control_frame = tk.Frame(root)
        self.control_frame.pack(pady=10)

        self.rtsp_listbox = tk.Listbox(self.control_frame, width=50, height=6)
        self.rtsp_listbox.pack(side=tk.LEFT)

        # Add and remove buttons
        self.add_button = tk.Button(self.control_frame, text="Thêm RTSP", command=self.add_rtsp_url)
        self.add_button.pack(side=tk.LEFT, padx=10)

        self.remove_button = tk.Button(self.control_frame, text="Xóa RTSP", command=self.remove_rtsp_url)
        self.remove_button.pack(side=tk.LEFT, padx=10)

        # Start button
        self.start_button = tk.Button(self.control_frame, text="Hiện Streams", command=self.start_streams)
        self.start_button.pack(side=tk.LEFT, padx=10)

        # Thread handling for streams
        self.stop_event = threading.Event()

        temp = pathlib.PosixPath
        pathlib.PosixPath = pathlib.WindowsPath
        # Load AI model
        self.model_path = './kaggle_weight_chicken/augment_best.pt'
        self.model = torch.hub.load('./yolov5', 'custom', path=self.model_path, source='local')

        # Hold references to labels and stop buttons for each stream
        self.labels = {}

        # Load RTSP URLs from file at startup
        self.load_rtsp_urls()
    def update_video_display(self):
        # Clear the current video display
        for widget in self.video_frame.winfo_children():
            widget.destroy()

        # Create new labels and checkboxes for each RTSP URL
        for index, rtsp_url in enumerate(self.rtsp_urls):
            label_original = tk.Label(self.video_frame, text=f"Video Gốc - {rtsp_url}")
            label_original.pack(anchor=tk.W)

            label_ai = tk.Label(self.video_frame, text=f"Video Qua AI - {rtsp_url}")
            label_ai.pack(anchor=tk.W)

            ai_var = tk.BooleanVar(value=self.ai_status[index])
            ai_checkbox = tk.Checkbutton(self.video_frame, text="Sử dụng AI", variable=ai_var,
                                         command=lambda idx=index: self.toggle_ai_status(idx, ai_var))
            ai_checkbox.pack(anchor=tk.W)

            # Stop button for the stream
            stop_button = tk.Button(self.video_frame, text="Dừng Stream", command=lambda idx=index: self.stop_stream(idx))
            stop_button.pack(anchor=tk.W)

            # Save directory button
            save_button = tk.Button(self.video_frame, text="Chọn thư mục lưu", command=lambda idx=index: self.choose_save_directory(idx))
            save_button.pack(anchor=tk.W)

            # Store references to the labels
            self.labels[rtsp_url] = (label_original, label_ai)
    def choose_save_directory(self, index):
    # Choose the main save directory
        save_path = filedialog.askdirectory(title="Chọn thư mục lưu video")
        if save_path:
            # Create a unique subfolder for the RTSP stream using the current timestamp
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            rtsp_subfolder = os.path.join(save_path, f"stream_{index}_{timestamp}")
            os.makedirs(rtsp_subfolder, exist_ok=True)  # Create the directory if it doesn't exist
            self.save_paths[index] = rtsp_subfolder

    def toggle_ai_status(self, index, ai_var):
        self.ai_status[index] = ai_var.get()  # Update AI status based on checkbox
        
    def add_rtsp_url(self):
        rtsp_url = simpledialog.askstring("Nhập RTSP URL", "Nhập URL RTSP:")
        if rtsp_url:
            self.rtsp_urls.append(rtsp_url)
            self.ai_status.append(False)  # Default to no AI detection
            self.stream_running.append(True)  # Start streams as running by default
            self.save_paths.append(None)  # Add placeholder for save path
            self.rtsp_listbox.insert(tk.END, rtsp_url)
            self.update_video_display()  # Update the display to include AI checkbox and stop button
            self.save_rtsp_urls()  # Save RTSP URLs to file

    def remove_rtsp_url(self):
        selected_index = self.rtsp_listbox.curselection()
        if selected_index:
            self.rtsp_urls.pop(selected_index[0])
            self.ai_status.pop(selected_index[0])  # Remove corresponding AI status
            self.stream_running.pop(selected_index[0])  # Remove corresponding stream status
            self.save_paths.pop(selected_index[0])  # Remove corresponding save path
            self.rtsp_listbox.delete(selected_index)
            self.update_video_display()
            self.save_rtsp_urls()  # Save RTSP URLs to file

    def load_rtsp_urls(self):
        """Load RTSP URLs from a file at the start of the application."""
        if os.path.exists(self.rtsp_file):
            with open(self.rtsp_file, 'r') as file:
                for line in file:
                    rtsp_url = line.strip()
                    if rtsp_url:
                        self.rtsp_urls.append(rtsp_url)
                        self.ai_status.append(False)  # Default to no AI detection
                        self.stream_running.append(True)  # Start streams as running by default
                        self.save_paths.append(None)  # Add placeholder for save path
                        self.rtsp_listbox.insert(tk.END, rtsp_url)
            self.update_video_display()

    def save_rtsp_urls(self):
        """Save the current list of RTSP URLs to a file."""
        with open(self.rtsp_file, 'w') as file:
            for rtsp_url in self.rtsp_urls:
                file.write(rtsp_url + '\n')

    def start_streams(self):
        for idx, (rtsp_url, ai) in enumerate(zip(self.rtsp_urls, self.ai_status)):
            if self.stream_running[idx]:
                threading.Thread(target=self.stream_video, args=(rtsp_url, ai, idx)).start()

    # Remaining methods like start_streams, stop_stream, stream_video, etc. stay the same
    
    def stop_stream(self, index):
        # Set the stream to not running, so it stops processing frames
        self.stream_running[index] = False
        # Clear the video frames displayed for this stream
        rtsp_url = self.rtsp_urls[index]
        self.labels[rtsp_url][0].config(image='')  # Clear original frame label
        self.labels[rtsp_url][1].config(image='')  # Clear AI frame label

    def stream_video(self, rtsp_url, use_ai, index):
        cap = cv2.VideoCapture(rtsp_url)

        # Ensure the save path exists
        if not self.save_paths[index]:
            print(f"No save path set for stream {index}")
            return

        # Retrieve the actual FPS of the RTSP stream
        actual_fps = 30
        target_fps = 10  # Target FPS
        skip_frames = int(actual_fps / target_fps) if actual_fps > target_fps else 1

        print(f"Actual FPS: {actual_fps}, Skip every {skip_frames} frame(s) to match {target_fps} FPS")

        # FFmpeg command to create HLS from raw video input
        m3u8_path = os.path.join(self.save_paths[index], f"output_{time.strftime('%Y%m%d-%H%M%S')}.m3u8")
        ffmpeg_command = [
            'ffmpeg',
            '-y',
            '-f', 'rawvideo',
            '-vcodec', 'rawvideo',
            '-pix_fmt', 'bgr24',  # Input pixel format (BGR from OpenCV)
            '-s', '1920x1080',  # Frame size fixed at 1080p
            '-r', str(target_fps),  # Frame rate (target FPS)
            '-i', '-',  # Input from stdin (OpenCV)
            '-c:v', 'libx264',  # Output codec (H.264)
            '-pix_fmt', 'yuv420p',  # Output pixel format
            '-preset', 'ultrafast',  # FFmpeg encoding preset
            '-tune', 'zerolatency',  # Minimize latency
            '-f', 'hls',  # Output format
            '-hls_time', '20',  # Segment length in seconds
            '-hls_list_size', '0',  # Keep all segments
            '-hls_flags', 'delete_segments',  # Auto-delete segments
            m3u8_path
        ]

        # Start FFmpeg process to create HLS
        process = subprocess.Popen(ffmpeg_command, stdin=subprocess.PIPE)

        frame_count = 0
        while not self.stop_event.is_set() and self.stream_running[index]:
            ret, frame = cap.read()
            if not ret:
                break

            # Skip frames to match the target FPS
            frame_count += 1
            if frame_count % skip_frames != 0:
                continue

            # Resize frame for consistency
            frame = cv2.resize(frame, (1920, 1080))

            # Apply AI processing and draw bounding boxes and text
            try:
                if use_ai:
                    # Run AI model on the frame
                    results = self.model(frame)
                    num_boxes = len(results.xyxy[0])

                    # Annotate and draw bounding boxes
                    cv2.putText(frame, f'So luong ga: {num_boxes}', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
                    for box in results.xyxy[0]:
                        x1, y1, x2, y2, conf, cls = box
                        if conf > 0.25:
                            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)

                # Write the annotated frame to FFmpeg stdin (to save it)
                process.stdin.write(frame.tobytes())

                # Display the frame (original or AI processed)
                if use_ai:
                    self.display_frame(frame, self.labels[rtsp_url][1])
                else:
                    self.display_frame(frame, self.labels[rtsp_url][0])

            except Exception as e:
                print(f"Error displaying frame for {rtsp_url}: {e}")

        # Release resources
        cap.release()
        process.stdin.close()
        process.wait()

    def display_frame(self, frame, label):
        # Convert frame to RGB for displaying in Tkinter
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)

        # Update the label with the new image
        label.config(image=imgtk)
        label.image = imgtk

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoStreamApp(root)
    root.mainloop()


YOLOv5  2024-9-16 Python-3.10.9 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Actual FPS: 30, Skip every 3 frame(s) to match 10 FPS


e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda

No save path set for stream 1


e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
e:\./yolov5\models\common.py:892: FutureWarning: `torch.cuda